- Pre requisites
    - Install Azure ML SDK v1.55 - recommended option is A) 
        - Option A) via ESML conda: Use conda/kernel in ESML: `azure_automl_esml_v155`, or kernel: `azure_automl_esml_v155_v155` (supports both SDK v1.55 and SDK v2 v 1.15)
            - How to install: https://github.com/jostrm/azure-enterprise-scale-ml/blob/main/environment_setup/user_dev_env_install/Readme_Install_VSCode_or_ComputeInstance.md
        - Option B) via Microsoft docs. Then you also need to pip install `%pip install -U python-dotenv`
            - https://learn.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py
    - Create an .env file, with the 3 variables: 
        - AZURE_CLIENT_ID - The client ID returned when you created the service principal.
        - AZURE_TENANT_ID - The tenant ID returned when you created the service principal.
        - AZURE_CLIENT_SECRET - The password/credential generated for the service principal.

## pip install Azure ML SDK v2 (v1.15) or not? 
- If you have installed: "azure-enterprise-scale-ml\environment_setup\user_dev_env_install\AzureML_v1_55_and_v2_1_15"
    Then you dont need to run below PIP
- If you have installed: "azure-enterprise-scale-ml\environment_setup\user_dev_env_install\AzureML_v1_55"
    - Then you need to run below PIP (3 cells)
        - azure-ai-ml, azure-identity, python-dotenv


%pip install azure-ai-ml

%pip install azure-identity

%pip install -U python-dotenv

In [ ]:
# TODO: CONFIGURE THIS
project_number="1"
region_short = "weu"
aifactory_environment = "dev" # dev,test,prod
aifactory_prefix = "ab-cbe"

sku_array = ["Standard_DS11_v2","STANDARD_NC24RS_V3", "Standard_D13_v2"]
compute_instance_sku = sku_array[0]
compute_instance_suffix = "01"
# TODO: CONFIGURE THIS

# NO TOUCH: Auto, since convention:
ai_factory_index = "001"
aml_instance  ="001"
subscription_id = "guid"
resource_group_name=aifactory_prefix+"-esml-project00"+project_number+"-"+region_short+"-"+aifactory_environment+"-"+ai_factory_index+"-rg"
resource_group_name_common=aifactory_prefix+"-esml-common-"+region_short+"-"+aifactory_environment+"-"+ai_factory_index
workspace_name="aml-prj00"+project_number+"-"+region_short+"-"+aifactory_environment+"-"+aml_instance
vnet_name = "vnt-esmlcmn-"+region_short+"-"+aifactory_environment+"-"+ai_factory_index # vnt-esmlcmn-uks-dev-001
subnet_name = "snet-esml-cmn-001"
vnetId =  "/"+subscription_id+"/resourceGroups/"+resource_group_name_common+"/providers/Microsoft.Network/virtualNetworks/"+vnet_name+"/subnets/"
subnet_fully_qualified = vnetId+subnet_name

In [ ]:
print(resource_group_name)
print(resource_group_name_common)
print(workspace_name)
print(vnet_name)
print(subnet_fully_qualified)

# Option A) Default credential

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential,InteractiveBrowserCredential

credential = InteractiveBrowserCredential()

#credential = DefaultAzureCredential()
# Check if given credential can get token successfully.
#credential.get_token("https://management.azure.com/.default")

ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group_name,
    workspace_name=workspace_name,
    force_token_refresh=True
)

# Option C) Service Principal login - with client secret in .env file
- Create an .env file with the below properties: 

AZURE_CLIENT_ID=aGuid

AZURE_TENANT_ID=aGuid

AZURE_CLIENT_SECRET=aSecret



# After option a,b or c the below LOGIN code can be used

In [ ]:
from azure.ai.ml import MLClient
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information to contain
    #       your subscription ID, resource group name, and workspace name
    client_config = {
        "subscription_id": subscription_id,
        "resource_group": resource_group_name,
        "workspace_name": workspace_name,
    }

    # write and reload from config file
    import json, os

    config_path = "../.azureml/config.json"
    os.makedirs(os.path.dirname(config_path), exist_ok=True)
    with open(config_path, "w") as fo:
        fo.write(json.dumps(client_config))
    ml_client = MLClient.from_config(credential=credential, path=config_path)

print(ml_client)

# DEFINE - Managed online endpoint

In [ ]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

## EP-name and Auth model

In [ ]:
# Example way to define a random name
import datetime
endpoint_name = "p"+project_number+"-"+datetime.datetime.now().strftime("%m%d%H%M%f")+ "-ep"+compute_instance_suffix 

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="this is a sample endpoint",
    auth_mode="key"
)

In [ ]:
option_a = True
model = None
env = None

if(option_a):
    model = Model(path="../model_diabetes/model.pkl")
    env = Environment(
        conda_file="../model_diabetes/conda.yaml",
        image="mcr.microsoft.com/azureml/curated/azureml-automl:159",
        name="diabetes-env")
else:
    model = Model(name="diabetes_automlsdk_mlflow")
    env = Environment(name="ESML-AzureML-155-AutoML_159_from_ctx")

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../model_diabetes", scoring_script="score.py"
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

# Check if it is alive

In [ ]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=False)
ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=False
)